Setup

In [1]:
from tensorflow_docs.vis import embed

from tensorflow import keras
from keras import layers

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
import random

Data preparation

In [2]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 960
# = 2048
IMG_SIZE = 224#128

#ROOT_VIDEOS = f"../Dataset/Frames/"
ROOT_VIDEOS = f"../Dataset/Noviembre2020/"
#ROOT_FEATURES = f"../Features/DenseNet121/FRAME"
#ROOT_FEATURES = f"../Features/DenseNet121_FRAME/"
#ROOT_FEATURES = f"../Features/ResNet101V2/FRAME/"
ROOT_FEATURES = f"../Features/MobileNetV3_Large/NORMAL_512/"

CNN

In [4]:
def build_feature_extractor_DenseNet():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

def build_feature_extractor_Inception():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

def build_feature_extractor_ResNet():
    feature_extractor = keras.applications.ResNet101V2(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.resnet_v2.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

def build_feature_extractor_MobileNet():
    feature_extractor = keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.mobilenet_v2.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

def build_feature_extractor_MobileNet3():
    feature_extractor = keras.applications.MobileNetV3Small(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.mobilenet_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

def build_feature_extractor_MobileNetLarge():
    feature_extractor = keras.applications.MobileNetV3Large(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.mobilenet_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")
#feature_extractor_DENSENET = build_feature_extractor_DenseNet()
#feature_extractor_INCEPTION = build_feature_extractor_Inception()
#feature_extractor_ResNet = build_feature_extractor_ResNet()
#feature_extractor_MobileNet = build_feature_extractor_MobileNet()

In [29]:
FEATURE_EXTRACTOR = build_feature_extractor_MobileNetLarge()

Load CSV

In [5]:
train_df = pd.read_csv("../video list/train.csv")
test_df = pd.read_csv("../video list/test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

Total videos for training: 80
Total videos for testing: 20


La clase 3.0 tiene 72 instancias

La clase 1.0 tiene 15 instancias

La clase 2.0 tiene 6 instancias

La clase 0.0 tiene 7 instancias


In [31]:
dataset_df = pd.read_csv("../video list/_lista_videos.csv")

print(dataset_df["Target"].value_counts())

Target
3.0    72
1.0    15
0.0     7
2.0     6
Name: count, dtype: int64


In [32]:
dataset_df = pd.read_csv("../video list/frames_list.csv")

print(dataset_df["Label"].value_counts())

Label
3.0    357
0.0    105
1.0     91
2.0     79
Name: count, dtype: int64


Methods for feature extractor

In [6]:
center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)

def ShowImage(frame):
    cv2.imshow('Frame', frame)
    cv2.waitKey(0)  # Esperar hasta que se presione una tecla
    cv2.destroyAllWindows()

def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze()
    return cropped

def get_dimension(res):
    res = res[1:-1].split(', ')

    right = int(res[0])
    up = int(res[1])
    left = int(res[2])
    down = int(res[3])
    return right, up, left, down

def load_video(path, resolution, max_frames=0):
    cap = cv2.VideoCapture(path)
    frames = []

    right, up, left, down = get_dimension(resolution)
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = frame[up:down, left:right]
            frame = crop_center(frame)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

def load_image(path):
    image = cv2.imread(path)
    frame = crop_center(image)
    frame = frame[:, :, [2, 1, 0]]
    return np.array(frame)
    #return cv2.resize(image, (IMG_SIZE, IMG_SIZE))

OS Methods

In [7]:
def Get_directory_amount(url):
    list_dir = os.listdir(url)
    amount = sum(os.path.isdir(os.path.join(url, x)) for x in list_dir)
    return amount

def Get_file_amount(url):
    list_dir = os.listdir(url)
    return len(list_dir)

Image Editor

In [8]:
def MostrarImagen(frame):
    cv2.imshow('Frame', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def AgregarRuidoGaussiano(dim, mean, variance):
    img = np.random.normal(mean, variance ** 0.5, dim).astype(np.uint8)
    img[:,:,1] = img[:,:,0]
    img[:,:,2] = img[:,:,0]
    #gaussian_noise = cv2.merge([gaussian_noise, gaussian_noise, gaussian_noise])
    return img

def AgregarRuidoGrises(img, p):
    values = [0, 50, 60, 70, 80]
    op = (1 - p) / 4
    probability = [p, op, op, op, op]  # Asegúrate de que sumen 1.0
    noise = np.random.choice(values, size=img.shape, p=probability).astype(np.uint8)
    noise[:,:,1] = noise[:,:,0]
    noise[:,:,2] = noise[:,:,0]
    return cv2.add(img, noise)

def GetRuidoGrises(dim, p):
    values = [0, 50, 60, 70, 80]
    op = (1 - p) / 4
    probability = [p, op, op, op, op]  # Asegúrate de que sumen 1.0
    noise = np.random.choice(values, size=dim, p=probability).astype(np.uint8)
    noise[:,:,1] = noise[:,:,0]
    noise[:,:,2] = noise[:,:,0]
    return noise

def RotarImagen(img, angle):
    height, width = img.shape[:2]
    center = (width // 2, height // 2)

    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    return cv2.warpAffine(img, rotation_matrix, (width, height))

def ZoomImagen(img, scale):    
    h, w = img.shape[:2]
    
    x = int((1-scale)*w/2.)
    y = int((1-scale)*h/2.)

    w = int(w * scale)
    h = int(h * scale)

    zoomed_roi = img[y:y+h, x:x+w]

    return cv2.resize(zoomed_roi, (img.shape[1], img.shape[0]))

def only_load_image(path):
    return cv2.imread(path)

def center_image(img):
    frame = crop_center(img)
    frame = frame[:, :, [2, 1, 0]]
    return np.array(frame)
    #return cv2.resize(img, (IMG_SIZE, IMG_SIZE))


In [36]:
def Load_1(path):    
    frame = only_load_image(path)
    print(frame.shape)
    frame = center_image(frame)
    print(frame.shape)
    MostrarImagen(frame)
    print(frame[None, :].shape)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    print(frame_features.shape)
    print(frame_features)

def Load_2(path):    
    frame = only_load_image(path)
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    MostrarImagen(frame)
    print(frame.shape)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    print(frame_features.shape)
    print(frame_features)
path = f'../Dataset/Frames/B1/Captura de Pantalla 2021-09-22 a la(s) 22.16.19.png'

Load_1(path)
Load_2(path)

(1046, 936, 3)
(224, 224, 3)
(1, 224, 224, 3)
(1, 960)
[[-7.42715131e-03 -3.35397711e-03 -0.00000000e+00 -3.18439193e-02
   2.02430353e-01 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -1.57069955e-02  1.68186080e+00
  -7.45083019e-03 -1.14069069e-02 -1.99164860e-02 -0.00000000e+00
  -3.90792033e-03 -4.60878424e-02 -0.00000000e+00 -8.61667679e-04
  -1.30856866e-02  2.65945345e-01  3.24167609e-01  1.46305120e+00
  -0.00000000e+00 -0.00000000e+00  2.32903528e+00 -0.00000000e+00
  -0.00000000e+00 -5.70655195e-03 -7.92278945e-02 -1.29416063e-02
  -0.00000000e+00 -2.93930601e-02 -0.00000000e+00  9.90219489e-02
   1.29497573e-01 -0.00000000e+00  4.68748951e+00  1.95307732e-02
  -0.00000000e+00 -1.35604211e-03 -5.35186287e-03  1.07141770e-02
  -3.76351643e-03 -1.86870750e-02  9.00641754e-02 -0.00000000e+00
  -1.79380298e-01 -1.04146063e-01 -1.22776501e-01 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
   3.05283904e+00 -0.

Feature Extractor

In [37]:
def prepare_all_videos(df):

    video_paths = df["Direccion"].values.tolist()
    video_resolution = df["Dimension"].values.tolist()
    labels = df["Target"].values
    labels = np.reshape(labels, (len(labels), 1))

    shortLabel = np.zeros(
        shape=(1, 1), dtype="float32"
    )
    finalLabels = np.zeros(
        shape=(0, 1), dtype="float32"
    )

    frame_masks = np.zeros(shape=(0, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(0, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    size = len(video_paths)
    
    for idx, path in enumerate(video_paths):
        inf = 0
        print("IDX: " + str(idx) + "/" + str(size) + "| path: " + path + "|inf:" + str(inf))

        video = load_video(ROOT_VIDEOS + path, video_resolution[idx])
        while inf < len(video):
            print("  inf: " + str(inf) + "|to: " + str(inf + MAX_SEQ_LENGTH))
            frames = video[inf:inf + MAX_SEQ_LENGTH]
            inf = inf + MAX_SEQ_LENGTH
        
            frames = frames[None, ...]

            temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
            temp_frame_features = np.zeros(
                shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
            )

            for i, batch in enumerate(frames):
                video_length = batch.shape[0]
                length = min(MAX_SEQ_LENGTH, video_length)
                for j in range(length):
                    temp_frame_features[i, j, :] = FEATURE_EXTRACTOR.predict(
                        batch[None, j, :], verbose=0
                    )
                temp_frame_mask[i, :length] = 1
   
            frame_features = np.concatenate((frame_features, temp_frame_features), axis=0)
            frame_features[idx,] = temp_frame_features.squeeze()
            
            frame_masks = np.concatenate((frame_masks, temp_frame_mask), axis=0)
            frame_masks[idx,] = temp_frame_mask.squeeze()

            shortLabel[0,0] = labels[idx, 0]
            finalLabels = np.concatenate((finalLabels, shortLabel), axis = 0)


    return (frame_features, frame_masks), finalLabels

In [38]:
train_data, train_labels = prepare_all_videos(train_df)
test_data, test_labels = prepare_all_videos(test_df)

IDX: 0/80| path: PAC32\pre (convertido).mov|inf:0
  inf: 0|to: 20
  inf: 20|to: 40
  inf: 40|to: 60
  inf: 60|to: 80
  inf: 80|to: 100
  inf: 100|to: 120
  inf: 120|to: 140
  inf: 140|to: 160
  inf: 160|to: 180
  inf: 180|to: 200
  inf: 200|to: 220
  inf: 220|to: 240
  inf: 240|to: 260
  inf: 260|to: 280
  inf: 280|to: 300
  inf: 300|to: 320
  inf: 320|to: 340
  inf: 340|to: 360
  inf: 360|to: 380
IDX: 1/80| path: PAC35\post.mov|inf:0
  inf: 0|to: 20
  inf: 20|to: 40


Exception ignored in: <function UniquePtr.__del__ at 0x000001D3A2F03130>
Traceback (most recent call last):
  File "c:\Users\Francisco\anaconda3\envs\py310\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 74, in __del__
    self.deleter(obj)
KeyboardInterrupt: 


  inf: 40|to: 60
  inf: 60|to: 80
  inf: 80|to: 100
  inf: 100|to: 120
  inf: 120|to: 140
  inf: 140|to: 160
  inf: 160|to: 180
IDX: 2/80| path: PAC46\pre.mov|inf:0
  inf: 0|to: 20
  inf: 20|to: 40
  inf: 40|to: 60
  inf: 60|to: 80
  inf: 80|to: 100


In [ ]:
def load_video_custom(path, resolution, max_frames=0):
    cap = cv2.VideoCapture(path)
    frames = []

    right, up, left, down = get_dimension(resolution)
    noise = None
    rotacion = random.uniform(-60, 60)
    zoom = random.uniform(0.85, 1.0)
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = frame[up:down, left:right]
            #_______________________________________________________________________
            if noise is None:
                noise = GetRuidoGrises(frame.shape, random.uniform(0.3, 0.7))                
            frame = cv2.add(frame, noise)
            # 
            frame = RotarImagen(frame, rotacion)
            frame = ZoomImagen(frame, zoom)
            #_______________________________________________________________________
            frame = crop_center(frame)
            frame = frame[:, :, [2, 1, 0]]

            #frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

def save_video(frame_features, label, path):
    #path = ROOT_FEATURES + path.split('/')[-1]
    #directory, file = os.path.split(path)
    #name, _ = os.path.splitext(file)
    name = str(Get_directory_amount(ROOT_FEATURES))
    directory = ROOT_FEATURES + "/" + name

    if not os.path.exists(directory):
        os.makedirs(directory)

    np.save(directory + "/data.npy", frame_features, fix_imports=True, allow_pickle=False)
    np.save(directory + "/label.npy", label, fix_imports=True, allow_pickle=False)

def save_video_modify(frame_features, label, path):
    path = ROOT_FEATURES + path.split('/')[-1]
    directory, file = os.path.split(path)
    name, _ = os.path.splitext(file)

    if not os.path.exists(directory):
        os.makedirs(directory)
    directory = directory + '/' + name
    if not os.path.exists(directory):
        os.makedirs(directory)

    num = int(Get_file_amount(directory) - 2)

    if (num < 0):
        np.save(f'{directory}/data.npy', frame_features, fix_imports=True, allow_pickle=False)
        np.save(f'{directory}/label.npy', label, fix_imports=True, allow_pickle=False)
    else:
        np.save(f'{directory}/data_m{num}.npy', frame_features, fix_imports=True, allow_pickle=False)

def prepare_all_individual_videos(df):
    
    video_paths = df["Direccion"].values.tolist()
    video_resolution = df["Dimension"].values.tolist()
    labels = df["Target"].values
    labels = np.reshape(labels, (len(labels), 1))

    count = np.zeros((4))

    size = len(video_paths)
    
    for idx, path in enumerate(video_paths):
        if (labels[idx] != 30):
        #if (labels[idx] == 2 or labels[idx] == 0):
            print("IDX: " + str(idx) + "/" + str(size) + "| path: " + path)

            frames = load_video_custom(ROOT_VIDEOS + path, video_resolution[idx])
            frame_features = np.zeros(
                shape=(frames.shape[0], NUM_FEATURES), dtype="float32"
            )

            frames = frames[None, ...]
            for i, batch in enumerate(frames):
                video_length = batch.shape[0]
                for j in range(video_length):
                    frame_features[j, :] = FEATURE_EXTRACTOR.predict(
                        batch[None, j, :], verbose=0)

            #save_video(frame_features, labels[idx], path)
            save_video_modify(frame_features, labels[idx], path)
            count[int(labels[idx])] += 1
    print(count)
    return

In [ ]:
#prepare_all_individual_videos(train_df, "train")
#prepare_all_individual_videos(test_df, "test")
prepare_all_individual_videos(dataset_df)

IDX: 0/100| path: PAC01\POST.mov
IDX: 1/100| path: PAC01\PRE.mov
IDX: 2/100| path: PAC02\PD.BASAL.R.POST.mov
IDX: 3/100| path: PAC02\PD.FINAL.RPOST.mov
IDX: 4/100| path: PAC03\PD.BASAL.R.POST.mov
IDX: 5/100| path: PAC03\PD.FINAL.RPOST.mov
IDX: 6/100| path: PAC04\PD.supino.Final.mov
IDX: 7/100| path: PAC04\PD.supino.mov
IDX: 8/100| path: PAC05\POST.mov
IDX: 9/100| path: PAC05\PRE.mov
IDX: 10/100| path: PAC06\POST.mov
IDX: 11/100| path: PAC06\PRE.mov
IDX: 12/100| path: PAC07\POST.mov
IDX: 13/100| path: PAC07\PRE.mov
IDX: 14/100| path: PAC08\POST.mov
IDX: 15/100| path: PAC08\PRE.mov
IDX: 16/100| path: PAC09\POST.mov
IDX: 17/100| path: PAC09\PRE.mov
IDX: 18/100| path: PAC10\POST.mov
IDX: 19/100| path: PAC10\PRE.mov
IDX: 20/100| path: PAC11\POST.mov
IDX: 21/100| path: PAC11\PRE.mov
IDX: 22/100| path: PAC12\POST.mov
IDX: 23/100| path: PAC12\PRE.mov
IDX: 24/100| path: PAC13\POST.mov
IDX: 25/100| path: PAC13\PRE.mov
IDX: 26/100| path: PAC14\POST.mov
IDX: 27/100| path: PAC14\PRE.mov
IDX: 28/100

In [ ]:
def save_features(frame_features, label, path, id):

    path = ROOT_FEATURES + path.split('/')[-1]
    directory, file = os.path.split(path)
    #name, _ = os.path.splitext(file)
    directory = directory + "/" + str(id)

    if not os.path.exists(directory):
        os.makedirs(directory)

    amount = Get_file_amount(directory)
    
    if (amount == 0):
        np.save(directory + "/data.npy", frame_features, fix_imports=True, allow_pickle=False)
        np.save(directory + "/label.npy", label, fix_imports=True, allow_pickle=False)
    else:
        np.save(f'{directory}/data_m{amount-2}.npy', frame_features, fix_imports=True, allow_pickle=False)

def prepare_all_frames(df):
    
    frames_paths = df["Url"].values.tolist()
    labels = df["Label"].values
    labels = np.reshape(labels, (len(labels), 1))

    size = len(frames_paths)
    
    for idx, path in enumerate(frames_paths):
        print("IDX: " + str(idx+1) + "/" + str(size) + "| path: " + path)

        frame = load_image(path)

        #frame_features = np.zeros(shape=(NUM_FEATURES), dtype="float32")

        frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)

        save_features(frame_features, labels[idx], path, idx)

#_________________________________________________________________________________
def SaveFrame_Original(path, label, idx):
    frame = only_load_image(path)
    frame = center_image(frame)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    save_features(frame_features, label, path, idx)

def SaveFrame_Noise(path, label, idx, noise):    
    frame = only_load_image(path)

    frame = AgregarRuidoGrises(frame, noise)

    frame = center_image(frame)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    save_features(frame_features, label, path, idx)

def SaveFrame_Rotation(path, label, idx, rotation):    
    frame = only_load_image(path)

    frame = RotarImagen(frame, rotation)

    frame = center_image(frame)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    save_features(frame_features, label, path, idx)

def SaveFrame_Zoom(path, label, idx, zoom):    
    frame = only_load_image(path)

    frame = ZoomImagen(frame, zoom)

    frame = center_image(frame)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    save_features(frame_features, label, path, idx)

def SaveFrame_Noise_Rotation(path, label, idx, noise, rotation):
    frame = only_load_image(path)

    frame = AgregarRuidoGrises(frame, noise)
    frame = RotarImagen(frame, rotation)

    frame = center_image(frame)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    save_features(frame_features, label, path, idx)
def SaveFrame_Noise_Zoom(path, label, idx, noise, zoom):
    frame = only_load_image(path)

    frame = AgregarRuidoGrises(frame, noise)
    frame = ZoomImagen(frame, zoom)

    frame = center_image(frame)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    save_features(frame_features, label, path, idx)
def SaveFrame_Rotation_Zoom(path, label, idx, rotation, zoom):
    frame = only_load_image(path)

    frame = RotarImagen(frame, rotation)
    frame = ZoomImagen(frame, zoom)

    frame = center_image(frame)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    save_features(frame_features, label, path, idx)
def SaveFrame_Noise_Rotation_Zoom(path, label, idx, noise, rotation, zoom):
    frame = only_load_image(path)

    frame = AgregarRuidoGrises(frame, noise)
    frame = RotarImagen(frame, rotation)
    frame = ZoomImagen(frame, zoom)
    
    frame = center_image(frame)
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    save_features(frame_features, label, path, idx)

#_________________________________________________________________________________

def prepare_all_frames_modify(df):    
    frames_paths = df["Url"].values.tolist()
    labels = df["Label"].values
    labels = np.reshape(labels, (len(labels), 1))

    size = len(frames_paths)
    
    for idx, path in enumerate(frames_paths):
        print("IDX: " + str(idx+1) + "/" + str(size) + "| path: " + path)
        
        SaveFrame_Original(path, labels[idx], idx)
        SaveFrame_Noise(path, labels[idx], idx, random.uniform(0.7, 0.95))

        SaveFrame_Rotation(path, labels[idx], idx, random.uniform(-5, 5))
        #SaveFrame_Zoom(path, labels[idx], idx, random.uniform(0.8, 0.9))

        SaveFrame_Noise(path, labels[idx], idx, random.uniform(0.2, 0.6))
        SaveFrame_Rotation(path, labels[idx], idx, random.uniform(-50, 50))
        #SaveFrame_Zoom(path, labels[idx], idx, random.uniform(0.4, 0.8))
        
        SaveFrame_Noise_Zoom(path, labels[idx], idx, random.uniform(0.7, 0.95), random.uniform(0.6, 0.9))
        SaveFrame_Noise_Zoom(path, labels[idx], idx, random.uniform(0.4, 0.7), random.uniform(0.6, 0.9))

        SaveFrame_Noise_Rotation(path, labels[idx], idx, random.uniform(0.8, 0.9), random.uniform(-5, 5))
        SaveFrame_Noise_Rotation(path, labels[idx], idx, random.uniform(0.4, 0.7), random.uniform(-40, 40))

        SaveFrame_Rotation_Zoom(path, labels[idx], idx, random.uniform(-4, 4), random.uniform(0.9, 1.0))
        SaveFrame_Rotation_Zoom(path, labels[idx], idx, random.uniform(-30, 30), random.uniform(0.9, 1.0))

        #SaveFrame_Noise_Rotation_Zoom(path, labels[idx], idx, random.uniform(0.7, 0.85), random.uniform(-7, 7), random.uniform(0.9, 1.0))
        #SaveFrame_Noise_Rotation_Zoom(path, labels[idx], idx, random.uniform(0.5, 0.65), random.uniform(-25, 25), random.uniform(0.9, 1.0))
        #SaveFrame_Noise_Rotation_Zoom(path, labels[idx], idx, random.uniform(0.2, 0.55), random.uniform(-40, 40), random.uniform(0.9, 1.0))

In [ ]:
prepare_all_frames(dataset_df)

KeyError: 'Url'

In [ ]:
path = f'../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.03.03.png'

frame = only_load_image(path)

frame = center_image(frame)
frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
print(frame_features.shape)

(1, 960)


In [ ]:
prepare_all_frames_modify(dataset_df)

IDX: 1/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.03.03.png
IDX: 2/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.05.30.png
IDX: 3/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.05.44.png
IDX: 4/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.09.54.png
IDX: 5/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.10.07.png
IDX: 6/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.10.30.png
IDX: 7/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.10.38.png
IDX: 8/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.10.52.png
IDX: 9/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.13.54.png
IDX: 10/632| path: ../Dataset/Frames/Normal/Captura de Pantalla 2021-09-22 a la(s) 22.22.29.png
IDX: 11/632| path: ../Dataset/Frames/Normal/Captu

KeyboardInterrupt: 

In [ ]:
print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"Frame labels in train set: {train_labels.shape}")
print()
print(f"Frame features in test set: {test_data[0].shape}")
print(f"Frame masks in test set: {test_data[1].shape}")
print(f"Frame labels in test set: {test_labels.shape}")

NameError: name 'train_data' is not defined

Data persistence

In [ ]:
np.save(f"../Features/train_data.npy", train_data[0], fix_imports=True, allow_pickle=False)
np.save(f"../Features/train_data_mask.npy", train_data[1], fix_imports=True, allow_pickle=False)
np.save(f"../Features/train_labels.npy", train_labels, fix_imports=True, allow_pickle=False)
np.save(f"../Features/test_data.npy", test_data[0], fix_imports=True, allow_pickle=False)
np.save(f"../Features/test_data_mask.npy", test_data[1], fix_imports=True, allow_pickle=False)
np.save(f"../Features/test_labels.npy", test_labels, fix_imports=True, allow_pickle=False)

In [ ]:
def LoadData(data, d, m, seq, stride, num_features):

    size = data.shape[0]
    i = stride
    lim = 0

    while (lim < size):
        lim = i + seq
        if (lim < size):
            d = np.append(d, [data[i:lim]], axis=0)
            m = np.append(m, [np.ones((seq))], axis=0)
        else:
            if ((size-i) / (lim-i) > 0.5):
                fix = np.zeros((lim - size, num_features))
                data = np.vstack((data, fix))

                m_ones = np.ones((size - i))
                m_ones = np.append(m_ones, np.zeros((lim - size)))

                d = np.append(d, [data[i:lim]], axis=0)
                m = np.vstack((m, m_ones))

        i = i + seq

    return d, m

def LoadVideo(dir, seq = 20, stride = 5, num_features = NUM_FEATURES):
    data = np.load(f'{dir}/data.npy')
    labels = np.load(f'{dir}/label.npy')

    d = np.zeros((0, seq, num_features))
    m = np.zeros((0, seq))

    i = 0
    while (i < int(seq/stride) - 1):
        d, m = LoadData(data, d, m, seq, i*stride, num_features)
        i = i + 1

    l = np.ones((d.shape[0], 1)) * labels[0]

    return d, m, l

data_, mask_, labels_ = LoadVideo('../Features/DenseNet121/PAC32/pre (convertido)/', seq = 60, stride = 4)

print(data_.shape)
print(mask_.shape)
print(labels_.shape)

(81, 60, 1024)
(81, 60)
(81, 1)


In [27]:
def guardar_video(path, dest, resolution, rate = 2):
    cap = cv2.VideoCapture(path)
    initIndex = Get_file_amount(dest)
    index = 0
    right, up, left, down = get_dimension(resolution)
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            '''if (index % rate == 0):
                frame = frame[up:down, left:right]
                url = f'{dest}/{initIndex}.jpg'
                initIndex += 1
                cv2.imwrite(url, frame)
            index += 1'''
            frame = frame[up:down, left:right]
            url = f'{dest}/{initIndex}.jpg'
            initIndex += 1
            cv2.imwrite(url, frame)
            return

    finally:
        cap.release()

def recorrerVideos(pathRoot, df):

    video_paths = df["Direccion"].values.tolist()
    video_resolution = df["Dimension"].values.tolist()
    labels = df["Target"].values
    rate = np.array((5,8,4,20))
    size = len(video_paths)
    
    for idx, path in enumerate(video_paths):
        inf = 0
        print("IDX: " + str(idx) + "/" + str(size) + "| path: " + path)
        l = (int)(labels[idx])
        dest = f'{pathRoot}/{l}'
        guardar_video(ROOT_VIDEOS + path, dest, video_resolution[idx], rate[l])
recorrerVideos('Frames', train_df)
#recorrerVideos('Frames', test_df)

IDX: 0/80| path: PAC32\pre (convertido).mov
IDX: 1/80| path: PAC35\post.mov
IDX: 2/80| path: PAC46\pre.mov
IDX: 3/80| path: PAC05\PRE.mov
IDX: 4/80| path: PAC21\POST.mov
IDX: 5/80| path: PAC07\PRE.mov
IDX: 6/80| path: PAC37\pre.mov
IDX: 7/80| path: PAC33\13.12 (convertido).mov
IDX: 8/80| path: PAC07\POST.mov
IDX: 9/80| path: PAC04\PD.supino.mov
IDX: 10/80| path: PAC34\pre.mov
IDX: 11/80| path: PAC22\POST.mov
IDX: 12/80| path: PAC14\POST.mov
IDX: 13/80| path: PAC16\PRE.mov
IDX: 14/80| path: PAC47\pre.mov
IDX: 15/80| path: PAC12\POST.mov
IDX: 16/80| path: PAC33\pre.mov
IDX: 17/80| path: PAC36\post.mov
IDX: 18/80| path: PAC02\PD.FINAL.RPOST.mov
IDX: 19/80| path: PAC34\post.mov
IDX: 20/80| path: PAC38\post.mov
IDX: 21/80| path: PAC43\post.mov
IDX: 22/80| path: PAC29\PRE.mov
IDX: 23/80| path: PAC28\POST.mov
IDX: 24/80| path: PAC03\PD.BASAL.R.POST.mov
IDX: 25/80| path: PAC50\post.mov
IDX: 26/80| path: PAC12\PRE.mov
IDX: 27/80| path: PAC29\POST.mov
IDX: 28/80| path: PAC27\PRE.mov
IDX: 29/80| 

In [3]:
import os

def CrearCarpeta(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

CrearCarpeta(f'Frames')
CrearCarpeta(f'Frames/0')
CrearCarpeta(f'Frames/1')
CrearCarpeta(f'Frames/2')
CrearCarpeta(f'Frames/3')